In [ ]:
pip install mtcnn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import glob
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import os
from PIL import Image
from keras.models import load_model
import tensorflow as tf
from time import time
import logging

In [ ]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)
print("height: %d"% h)
print("width: %d"% w)

In [ ]:
# data = np.expand_dims(X.reshape((1288,50,37)),axis = -1)
data = []
label = []
for i in range(len(X)):
    gray = cv2.resize(X[i],(32,32))
    backtorgb = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)
    data.append(backtorgb)
    label.append(y[i])
data = np.array(data)
label = np.array(label).reshape((-1,1))
data.shape, label.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data, label, test_size=0.25, random_state=42)
X_train.shape,y_test.shape,y_train.shape, y_test.shape

In [ ]:
base_model = tf.keras.applications.VGG16( include_top = False, input_shape = (32,32,3))
for layer in base_model.layers[:-1]: 
    layer.trainable = True
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dense(128, activation='relu')(x)
preds = tf.keras.layers.Dense(7, activation ='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)
print(model.summary())

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(y_train)

y_train = lb.transform(y_train)

y_test = lb.transform(y_test)
y_train.shape

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train,y_train, epochs = 100)

pred = model.predict(X_test)
predictions = lb.inverse_transform(pred)
y_true = lb.inverse_transform(y_test)

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

plt.figure()
plt.plot(acc, label='Training Accuracy')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')

plt.figure()

plt.plot(loss, label='Training Loss')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.xlabel('epoch')
plt.grid()
plt.show()


In [ ]:
from sklearn.metrics import classification_report ,confusion_matrix ,plot_confusion_matrix
print(classification_report(y_true,predictions, target_names=target_names))